<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/14-WordEmbedings/Embedings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Burada RAG için önemli olacak bir yapıyı inceliyoruz text içinden konu ile alakalı şeyleri bulmak için bir text yöentim sistemi

Yazı → Embedding → Cosine Similarity → En alakalı içerikler → LLM'e ver → Süper tahmin


# Pre-Req


In [ ]:
%pip install -q sentence-transformers scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.0 MB/s eta 0:00:00


# En Basit Hafıza Modelimiz

memory = Hem metin hem embedding saklıyoruz.

model = Ücretsiz, hafif bir encoder.

MAX_MEMORY = Hafızamız dolunca eskileri atıyoruz.

In [ ]:
# 1. Gerekli modülleri alalım
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 2. Embedding modelimizi yükleyelim (çok hafif bir model)
model = SentenceTransformer('all-MiniLM-L6-v2')

# 3. Haber hafızamızı tutacak yer
memory = []

# 4. Sliding window kapasitesi
MAX_MEMORY = 5  # son 5 haber tutulacak

# Haberi / Yeni Metini Kaydetme Fonksiyonu

In [ ]:
def add_news_to_memory(text):
    # Haber embed ediliyor (sayılara çeviriliyor)
    embedding = model.encode([text])[0]

    # Memory'ye kaydediyoruz
    memory.append({
        "text": text,
        "embedding": embedding
    })

    # Sliding window: Hafıza dolarsa en eskiyi sil
    if len(memory) > MAX_MEMORY:
        memory.pop(0)
